In [ ]:
%cd ..

In [ ]:
import pandas as pd
from src.read_pdfs_v2 import process_applications
from src.preprocess_table_inmi import preprocess_applications_df
from src.model_training import train_model

In [ ]:
df_grant_list = pd.read_csv('data_headers/header_Grant.csv')
df_ref_list = pd.read_csv('data_headers/header_Refusal.csv')

headers_list = list(set(df_grant_list.super_header.tolist() + df_ref_list.super_header.tolist()))

type_case = 'Grant'
##list_headers_g = scan_folder_and_save_csv('data_all/Info Students/'+type_case, 'header_'+type_case+'.csv', check_folders = 500)
df_g = process_applications('data_all/Info Students/'+type_case,headers_list, check_folders = None)

type_case = 'Refusal'
#list_headers_r = scan_folder_and_save_csv('data_all/Info Students/'+type_case, 'header_'+type_case+'.csv',check_folders = 300)
df_r = process_applications('data_all/Info Students/'+type_case,headers_list, check_folders = None)

In [ ]:
df_g['type_case'] = 'granted'
df_r['type_case'] = 'refused'
df_cases = pd.concat([df_g,df_r])
df_cases.to_csv('data_scripts/data_onevisa_s.csv', index=True)
df_cases.shape

In [ ]:
df_cases = pd.read_csv('data_scripts/data_onevisa_s.csv').set_index('sample_id')

In [ ]:
df_preprocess = preprocess_applications_df(df_cases)

In [ ]:
df_preprocess = df_preprocess.reset_index()

In [ ]:
model_xgb, df_pred_xgb = train_model(df_preprocess, model_type ='xgboost',
                                     target_col='type_case', threshold=0.25)
